In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import time
from tqdm import tqdm
from IPython.display import Image
import regex as re
from IPython.core.display import display,HTML
from joblib import Parallel, delayed
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
PATH=ChromeDriverManager().install()

In [15]:
url = 'https://www.acunr.es/perros/'
driver=webdriver.Chrome(PATH) 
driver.get(url)

links= []

for e in tqdm(range(5)):
    muchos = driver.find_element(By.CLASS_NAME, 'col-lg-12')
    uno = muchos.find_elements(By.TAG_NAME, 'a')
    links += (e.get_attribute('href') for e in uno)
    otra = driver.find_element(By.XPATH, '//*[@id="main"]/ul[1]/li[8]/a')
    otra.click()
'''
ESTO NO LO PONGO DENTRO DEL BUCLE PORQUE ES LA ULTIMA PAG
SI LO PONGO DENTRO DEL BUCLE, INTENTA DAR A CLICK Y ME DA ERROR
PORQUE NO HAY OTRA PAG
'''
muchos = driver.find_element(By.CLASS_NAME, 'col-lg-12')
uno = muchos.find_elements(By.TAG_NAME, 'a')
links += [e.get_attribute('href') for e in uno]


links_acunr= []
for e in range(len(links)):
    if e % 2:
        links_acunr.append(links[e])
    else:
        pass
    
    

imagen=[]
nombres= []
carac= []
protec_id=[]
for e in tqdm(links_acunr):
        protec_id.append(2) 
        driver.get(e)
        time.sleep(0.2)
        nombres.append(driver.find_element(By.TAG_NAME, 'h2').text)
        box = driver.find_element(By.CLASS_NAME, 'col-lg-12')
        img= box.find_element(By.TAG_NAME, 'img')
        imagen.append(img.get_attribute('src'))
        desc = box.find_elements(By.CLASS_NAME, 'card-text')
        for i in desc:
            if len(desc) == 12:
                desc.insert(9, i)
            else:
                pass
            
            i = i.text
            i = i.split(':', 1)
            i.pop(0)
            carac += i
            
            
            
driver.quit()

n = 13
caracter = [carac[i:i + n] for i in range(0, len(carac), n)]

  2%|█▌                                                                                 | 1/52 [00:08<06:52,  8.10s/it]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x0108ACD3+2075859]
	Ordinal0 [0x0101EE61+1633889]
	Ordinal0 [0x00F1B7BD+571325]
	Ordinal0 [0x00F02E1A+470554]
	Ordinal0 [0x00F6AA0B+895499]
	Ordinal0 [0x00F7AC96+961686]
	Ordinal0 [0x00F67136+880950]
	Ordinal0 [0x00F3FEFD+720637]
	Ordinal0 [0x00F40F3F+724799]
	GetHandleVerifier [0x0133EED2+2769538]
	GetHandleVerifier [0x01330D95+2711877]
	GetHandleVerifier [0x0111A03A+521194]
	GetHandleVerifier [0x01118DA0+516432]
	Ordinal0 [0x0102682C+1665068]
	Ordinal0 [0x0102B128+1683752]
	Ordinal0 [0x0102B215+1683989]
	Ordinal0 [0x01036484+1729668]
	BaseThreadInitThunk [0x75B06939+25]
	RtlGetFullPathName_UEx [0x77DD8FD2+1218]
	RtlGetFullPathName_UEx [0x77DD8F9D+1165]


In [8]:
colnames= ('ID', 'EDAD', 'RAZA', 'SEXO', 'ORIGEN', 'CARACTER', 'APTO_CON_GATOS', 'LEHISMANIA', '4DX', 'SALUD', 'SITUACION', 'TAMAÑO', 'OBSERVACIONES')
perros_2 = pd.DataFrame(caracter, columns=colnames)
perros_2['PROTECTORA']=protec_id
perros_2['NOMBRE']= nombres
perros_2['LINK']= links_acunr

images1 = imagen
perros_2['FOTO'] = images1

# convert your links to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

pd.set_option('display.max_colwidth', None)

image_cols = ['FOTO']  #<- define which columns will be used to convert to html

# Create the dictionariy to be passed as formatters
format_dict = {}
for e in image_cols:
    format_dict[e] = path_to_image_html


# display(HTML(acunr_ini.to_html(escape=False ,formatters=format_dict)))


años= []
for e in perros_2.EDAD:
    e = e.lower()
    if ' adulto' in e or ' senior' in e or ' yayo' in e:
        años.append(int(re.findall('[0-9]+', e)[0]))
    elif ' cachorro' in e:
        años.append(0)
    else:
        años.append(e)
perros_2['AÑOS'] = años

raza= []
for e in perros_2.RAZA:
    e = e.lower()
    e= e.strip()
    raza.append(e)
perros_2['RAZA']= raza

sexo= []
for e in perros_2.SEXO:
    e = e.lower()
    e= e.strip()
    sexo.append(e)
perros_2['SEXO']= sexo

tam= []
for e in perros_2.TAMAÑO:
    e = e.lower()
    e= e.strip()
    tam.append((re.findall('[a-z,ñ]+', e)[0]))
perros_2['TAMAÑO']= tam

nom= []
for e in perros_2.NOMBRE:
    e = e.capitalize()
    e= e.strip()
    nom.append(e)
perros_2['NOMBRE']= nom


perros_2.drop(['EDAD', 'ID', 'ORIGEN', 'APTO_CON_GATOS', 'LEHISMANIA', '4DX', 'SALUD', 'SITUACION', 'OBSERVACIONES'], axis= 1, inplace = True)

In [9]:
perros_2.to_csv('perros_2.csv')

In [25]:
url = 'https://www.acunr.es/gatos/'
driver=webdriver.Chrome(PATH) 
driver.get(url)

links= []
muchos = driver.find_element(By.CLASS_NAME, 'col-lg-12')
uno = muchos.find_elements(By.TAG_NAME, 'a')
links += (e.get_attribute('href') for e in uno)

links_acunr= []
for e in range(len(links)):
    if e % 2:
        links_acunr.append(links[e])
    else:
        pass
    
imagen=[]
nombres= []
carac= []
protec_id=[]
for e in tqdm(links_acunr):
        protec_id.append(2) 
        driver.get(e)
        time.sleep(0.2)
        nombres.append(driver.find_element(By.TAG_NAME, 'h2').text)
        box = driver.find_element(By.CLASS_NAME, 'col-lg-12')
        img= box.find_element(By.TAG_NAME, 'img')
        imagen.append(img.get_attribute('src'))
        desc = box.find_elements(By.CLASS_NAME, 'card-text')
        for i in desc:
            if len(desc) == 9:
                desc.insert(7, i)
            else:
                pass
            
            i = i.text
            i = i.split(':', 1)
            i.pop(0)
            carac += i
            
n = 10
caracter = [carac[i:i + n] for i in range(0, len(carac), n)]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:21<00:00,  3.55s/it]


In [33]:
colnames= ('ID', 'EDAD', 'SEXO', 'ORIGEN', 'CARACTER', 'LEUCEMIA', 'INMUNO', 'SALUD', 'SITUACION', 'OBSERVACIONES')
gatos_2 = pd.DataFrame(caracter, columns=colnames)
gatos_2['PROTECTORA']=protec_id
gatos_2['NOMBRE']= nombres
gatos_2['LINK']= links_acunr

images1 = imagen
gatos_2['FOTO'] = images1

# convert your links to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

pd.set_option('display.max_colwidth', None)

image_cols = ['FOTO']  #<- define which columns will be used to convert to html

# Create the dictionariy to be passed as formatters
format_dict = {}
for e in image_cols:
    format_dict[e] = path_to_image_html


# display(HTML(acunr_ini.to_html(escape=False ,formatters=format_dict)))


años= []
for e in gatos_2.EDAD:
    e = e.lower()
    if ' adulto' in e:
        años.append(int(re.findall('[0-9]+', e)[0]))
    elif ' cachorro' in e:
        años.append(0)
    else:
        años.append(e)
gatos_2['AÑOS'] = años


sexo= []
for e in gatos_2.SEXO:
    e = e.lower()
    e= e.strip()
    sexo.append(e)
gatos_2['SEXO']= sexo


nom= []
for e in gatos_2.NOMBRE:
    e = e.capitalize()
    e= e.strip()
    nom.append(e)
gatos_2['NOMBRE']= nom

gatos_2.drop(['ID', 'EDAD', 'ORIGEN', 'LEUCEMIA', 'INMUNO', 'SALUD', 'SITUACION', 'OBSERVACIONES'], axis= 1, inplace = True)

In [36]:
gatos_2.to_csv('gatos_2.csv', index= False)